## Here, the goal is I want to drive the Fluxonium qubit through its flux line and I don't know yet what will happen. 

In [11]:
import scqubits as scq
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
import scipy.optimize as opt
from qutip.qip.operations import cnot
import json

# Load parameters from JSON
with open("cr_params.json", "r") as f:
    params = json.load(f)
FT_Device = params["FUN5_Right"]
# FT_Device = params["UMA5"]
# FT_Device = params["FTF_FUN10_Type_1"]

k = 120
trunc = 6

Fl1 = scq.Fluxonium(
    EJ=FT_Device['fluxonium']['EJ'],
    EC=FT_Device['fluxonium']['EC'],
    EL=FT_Device['fluxonium']['EL'],
    flux=0.5,
    cutoff=k,
    truncated_dim=trunc
)
Tr = scq.Transmon(
    EJ=FT_Device['transmon']['EJ'],
    EC=FT_Device['transmon']['EC'],
    ng=0,
    ncut=k,
    truncated_dim=trunc
)

J1T = FT_Device['Jc']

full_system = scq.HilbertSpace([Fl1, Tr])
full_system.add_interaction(
    g=J1T,
    op1=(Fl1.n_operator(), Fl1),
    op2=(Tr.n_operator(), Tr)
)
full_system.generate_bare_esys()
full_system.generate_lookup()

H_full = 2 * np.pi * full_system.hamiltonian()
evals, ekets = H_full.eigenstates()

n_Fl1_full = scq.identity_wrap(Fl1.n_operator(), Fl1, [Fl1, Tr])
n_Tr_full = scq.identity_wrap(Tr.n_operator(), Tr, [Fl1, Tr])

phi_Fl1_full = scq.identity_wrap(Fl1.phi_operator(), Fl1, [Fl1, Tr])
# phi_Tr_full = scq.identity_wrap(Tr.phi_operator(), Tr, [Fl1, Tr])

def get_dressed_epairs(evals, ekets):
    dressed_epairs = np.empty((5, 5), dtype=object)
    for i in range(4):
        for j in range(4):
            index = full_system.dressed_index((i, j))
            ztemp = qt.tensor(qt.basis(trunc, i), qt.basis(trunc, j)).overlap(ekets[index])
            np.angle(ztemp)
            dressed_epairs[i, j] = (evals[index], np.exp(-1j * np.angle(ztemp)) * ekets[index])
    return dressed_epairs

dressed_epairs = get_dressed_epairs(evals, ekets)

In [12]:
phi_F_00_01 = phi_Fl1_full.matrix_element(dressed_epairs[0, 0][1], dressed_epairs[0, 1][1])
phi_F_10_11 = phi_Fl1_full.matrix_element(dressed_epairs[1, 0][1], dressed_epairs[1, 1][1])
# phi_T_00_01 = phi_Tr_full.matrix_element(dressed_epairs[0, 0][1], dressed_epairs[0, 1][1])
# phi_T_10_11 = phi_Tr_full.matrix_element(dressed_epairs[1, 0][1], dressed_epairs[1, 1][1])
print("-i * <00|n_Fl1|01> =", -1j * phi_F_00_01)
print("-i * <10|n_Fl1|11> =", -1j * phi_F_10_11)
# print("-i * <00|n_Tr|01> =", -1j * phi_T_00_01)
# print("-i * <10|n_Tr|11> =", -1j * phi_T_10_11)

-i * <00|n_Fl1|01> = (0.004804163117197348+2.9449416724434294e-15j)
-i * <10|n_Fl1|11> = (-0.042598467576720284+1.8118144259506995e-16j)
